In [23]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from numpy import random
import time 
import copy 
import csv

In [24]:

def load_csv_with_counter(file_name):
    row_counter = 0
    rowwise_lists = []

    # Open and read the CSV file
    with open(file_name, mode='r') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Skip header row

        # Iterate through each row in the CSV file
        for row in reader:
            row_as_int = [int(value) for value in row]  # Convert each value to integer
            rowwise_lists.append(row_as_int)
            row_counter += 1  # Increment the counter
    return rowwise_lists

file_name = "C:\\Intern\\Gittin\'s plots\\All codes directory\\Fig 3 B,C (Mono HR)\\jobsize_data_incHR.csv"  # Replace with the actual file name
rowwise_lists = load_csv_with_counter(file_name)


In [25]:
def calculate_v_values(Q_values_action1, Q_values_action2):
    V_values = np.zeros((100,9))
    for i in range(100):
        for j in range(9):
            V_values[i][j] = max(Q_values_action1[i][j],Q_values_action2[i][j])  # Use keepdims=True to maintain the 2D shape
    return V_values

V_true = np.zeros((100,9))
# V_true = [9, 8.15241315, 7.47595723, 6.90163656, 6.49113047]

def bellman_relative_error(V_approx, V_true):
    nonzero_indices = V_true != 0
    if np.any(nonzero_indices):
        #relative_errors = np.abs((V_approx[nonzero_indices] - V_true[nonzero_indices]) / V_true[nonzero_indices])
        relative_errors = np.abs((V_approx[nonzero_indices] - V_true[nonzero_indices]))
        return np.mean(relative_errors)
    else:
        return np.nan

In [26]:
V_true[1,:] = [18.39001075, 24.63113997, 28.37029607, 32.74919392, 36.6070096,  39.95416782,
  42.26659913, 45.21643186, 47.6777546 ]
V_true[2,:] =  [28.20408841, 38.31997852, 43.99755973, 52.84045075, 60.77817213, 69.34942784,
  74.86945361, 83.47013721, 92.19408469]
V_true[3,:] =  [23.39255981, 33.64927941, 41.51103384, 49.29737536, 60.0681885,  65.63925207,
  73.01061097, 78.24495162, 73.41496629]
V_true[4,:] =  [20.27661416, 31.27369285, 41.65217067, 48.66837802, 54.16685682, 57.77537873,
  54.99415751, 38.90083371, 13.70105255]



In [27]:
V_true[1,:] = [ 2.88866783e+01,  3.15085928e+01,  3.40536870e+01,  3.57302799e+01,
   3.86458610e+01,  4.02700595e+01,  4.33413655e+01,  4.58000668e+01,
   4.77588016e+01]
V_true[2,:] =  [ 4.10888854e+01 , 4.65526320e+01,  5.26920938e+01,  5.59839147e+01,
   6.45651019e+01,  6.85675634e+01,  7.75534687e+01,  8.56410058e+01,
   9.25763321e+01]
V_true[3,:] =  [ 5.08628628e+01,  5.63930518e+01,  5.98191169e+01,  6.61043434e+01,
   7.16273587e+01,  7.58738044e+01,  8.24998123e+01,  8.61913750e+01,
   8.50192653e+01]
V_true[4,:] =  [ 6.24605652e+01,  6.27808790e+01,  6.82740106e+01,  7.22510249e+01,
   7.60514963e+01,  7.29368165e+01,  6.19245576e+01,  4.28895914e+01,
   1.27635876e+01]

In [30]:
def get_prob(step,p1,lamda):
    pi = [0]*step
    for i in range(1,step):
        pi[i]=(1-(1-p1)*((lamda)**(i)))
    mul = pi[step-1]
    return mul 

class envir():
    def __init__(self):
        self.phi = 9
        self.lamda = 0.8 
        self.p1=[]
        for i in range(9):
            self.p1.append(0.1*(i+1))

    
    def step(self,s,task,jobsize):
            s_old = s[task]
            next_state = copy.copy(s)
            reward = [0]*9
            if (s[task] == jobsize[task] or s[task] == 0):
                next_state[task] = 0
            else:
                #print(s,task,get_prob(s[task],self.p1[task],self.lamda))
                next_state[task]+= 1 
            
            if s[task]!=0 and next_state[task] == 0:
                reward[task] = 1
            elif s[task]==0:
                reward[task] = -10000
            else:
                reward[task] = 0
            return next_state,reward[task]
        
    def getjobsize(self, job_counter):
        jobsize = rowwise_lists[job_counter]
        return jobsize


class Agent():
    def __init__(self,alpha,gamma):
        self.Q_values = np.zeros((100,100,9))
        self.alpha = alpha
        #self.c= c
        self.gamma = gamma
        
    def activate_task_eps_greedy(self,s,M,epsilon):
            wl= []
            p = np.array([0,0,0,0,0,0,0,0,0])
            if np.random.random() < epsilon:
                for i in range(len(p)):
                    if s[i]!=0:
                        wl.append(i)
                arm_to_pull = np.random.choice(wl,1)[0]
                p[arm_to_pull] = 1
                return arm_to_pull
            else:
                p2 = {}
                for i in range(len(p)):
                    if s[i]!=0:
                        p2[i] = M[s[i]][i]
                max_key = max(p2, key=p2.get)
                p[max_key] = 1
                return max_key
    
    def check_best_action(self,state):
        inds = np.argmax(state)
        all_inds = []
        for i in range(len(state)):
            if state[inds] == state[i]:
                all_inds.append(i)
        return all_inds


In [ ]:
from tqdm import tqdm
def select_task(s,M):
        k = []
        for i in range(9):
            k.append(M[s[i]][i])
        max_index = np.argmax(k)  # Get the index of the maximum value
        return max_index
class game():
    start_time = time.time()
    t = 0
    M = np.zeros((100,9))
    Q_values = np.zeros((100,100,9))
    c0 = 0
    hist01 = [[] for i in range(9)]
    ti = []
    c1 = 0
    m = 0
    m1 = 0
    ct = 0
    cumm_rew = []
    BRE = []
    plt_wrong_actions = []
    cumm_wrong_steps = []
    eps = 1
    F = np.zeros((100,9))
    env = envir()
    episodes = 2500
    agent = Agent(alpha = 0.3,gamma = 0.99)
    for i in tqdm(range(episodes)):
        s = np.array([1,1,1,1,1,1,1,1,1])
        t = t+1
        eps = eps*0.9985
        #eps = eps*0.9996
        #Calculating Q values over state space for a given arm for the given M vector through Q learning
        #for step in range(2):
        episode_rew = 0
        jobsize = env.getjobsize(i)
        while (s[0]!=0 or s[1]!=0 or s[2]!=0 or s[3]!=0 or s[4]!=0 or s[5]!=0 or s[6]!=0 or s[7]!=0 or s[8]!=0):
            if len(cumm_rew)==0:
                cumm_rew.append(episode_rew)
            else:
                cumm_rew.append(cumm_rew[-1]+episode_rew)
            #print(s)
            task_eps = agent.activate_task_eps_greedy(s,M,eps)
            task_opt = agent.check_best_action(s)
            task = task_eps
            #task = select_task(s,M)
            current_time = time.time()-start_time
            next_state, R = env.step(s,task,jobsize)
            episode_rew+=R
            if task_eps not in task_opt:
                cumm_wrong_steps.append(1)
            else:
                cumm_wrong_steps.append(0)
            plt_wrong_actions.append(np.mean(cumm_wrong_steps)*100)
            Q0 = np.zeros((100,9))
            for i in range(100):
                for j in range(9):
                    Q0[i][j] = Q_values[i][i][j]
            V_values = calculate_v_values(Q0,M)
            BRE.append(bellman_relative_error(V_values,V_true))

            #print(task)
            #print(next_state)
            #print('r',R)
            for k in range(10):
                Q_values[s[task]][k][task] += agent.alpha*(R+agent.gamma*(max(M[k][task],copy.copy(Q_values[next_state[task]][k][task])))-Q_values[s[task]][k][task])    
            if((t%100)==0):
                agent.alpha = agent.alpha-0.000002
            s= copy.copy(next_state) 

          #Algorithm for stochastic approximation

          #Calculating F
            for i in range(10):
                F[i][task] = -M[i][task]+Q_values[i][i][task]
                #print('F',F[i][task])
                #print('m',M[i][task])
                #print('q',Q_values[i][i][task])

          #Print values
            #print("Iteration no",t)
            #print("F",F)

          #Update M
            for i in range(10):
                M[i][task] = M[i][task] + 0.2*(F[i][task])       
            for l in range(9):
                hist01[l].append(0.01*M[4][l])
            ti.append(current_time)
          #Stopping criteria
            
          #For M[i]
            '''if(t>=10):
                if(F[0]==0 and (histV0[len(histV0)-1]-histV0[len(histV0)-5]<0.1)):
                    ct=1;
                    m=M[0]   
          #For algorithm
            print(" ")
            histm0.append(M[0])
            histm1.append(M[1])
            if ((np.linalg.norm(F)<.9) or t>=9000) and (t>=2 and np.linalg.norm(V-Vold)<0.1):
                print("Gittin's index for state 0 is",0.2*min(m,(M[0])))
                print("Gittin's index for state 1 is",0.2*(M[1]))
                break'''
    v_values = calculate_v_values(Q0,M)
    print(M)
    print(hist01[:5])
    for j in range(9):
        plt.title('Gittins index of state 4 of job 2 vs runtime plot',fontsize='xx-large')
        plt.xlabel('Time', fontsize = 'xx-large')
        plt.ylabel('Gittins index',fontsize = 'xx-large')
        plt.plot(hist01[j],'-',c='red')
    plt.show()

    print(V_values)
    filename = 'C:\\Intern\\percent_wrong_QGI_incHR.csv'

    # Writing to CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['percent_wrong'])  # Writing the header
        for value in plt_wrong_actions:
            writer.writerow([value])  # Writing each value in a new row

    filename = 'C:\\Intern\\BRE_QGI_incHR.csv'

    # Writing to CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['BRE'])  # Writing the header
        for value in BRE:
            writer.writerow([value])  # Writing each value in a new row

    filename = 'C:\\Intern\\cumm_rew_QGI_incHR.csv'
    
    # Writing to CSV file
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['cumm_rew'])  # Writing the header
        for value in cumm_rew:
            writer.writerow([value])  # Writing each value in a new row     
            

  0%|          | 0/2500 [00:00<?, ?it/s]

 52%|█████▏    | 1289/2500 [00:56<01:15, 16.10it/s]